In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding as asym_padding
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

def client_hello():
    return os.urandom(16).hex()

def server_hello():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()

    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return {
        'hello': os.urandom(16).hex(),
        'public_key': public_pem,
        'private_key': private_key
    }

def send_premaster_secret(server_public_key_pem):
    server_public_key = serialization.load_pem_public_key(server_public_key_pem)
    premaster_secret = os.urandom(32)
    encrypted_premaster = server_public_key.encrypt(
        premaster_secret,
        asym_padding.OAEP(
            mgf=asym_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_premaster

def generate_session_keys(encrypted_premaster, private_key):
    premaster_secret = private_key.decrypt(
        encrypted_premaster,
        asym_padding.OAEP(
            mgf=asym_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return {'session_key': os.urandom(32)}

def encrypt_with_session_key(session_key, message):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(session_key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    padder = sym_padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(message.encode()) + padder.finalize()
    encrypted = encryptor.update(padded_data) + encryptor.finalize()
    return iv, encrypted

def decrypt_with_session_key(session_key, encrypted, iv):
    cipher = Cipher(algorithms.AES(session_key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    unpadder = sym_padding.PKCS7(algorithms.AES.block_size).unpadder()
    decrypted_padded = decryptor.update(encrypted) + decryptor.finalize()
    decrypted = unpadder.update(decrypted_padded) + unpadder.finalize()
    return decrypted


def main():
    print("Initiating TLS/SSL handshake")

    client_hello_message = client_hello()
    print("Client Hello:", client_hello_message)

    server_response = server_hello()
    print("Server Hello:", server_response['hello'])

    encrypted_premaster = send_premaster_secret(server_response['public_key'])
    print("Encrypted Premaster Secret from client:", encrypted_premaster.hex())

    session_keys = generate_session_keys(encrypted_premaster, server_response['private_key'])
    print("Session Key:", session_keys['session_key'].hex())

    client_iv, client_ready = encrypt_with_session_key(session_keys['session_key'], 'ready')
    server_iv, server_ready = encrypt_with_session_key(session_keys['session_key'], 'ready')
    print("Client Ready:", client_ready.hex())
    print("Server Ready:", server_ready.hex())

    test_message = "Test Message"
    enc_iv, encrypted_message = encrypt_with_session_key(session_keys['session_key'], test_message)
    print("Encrypted message:", encrypted_message.hex())

    decrypted_message = decrypt_with_session_key(session_keys['session_key'], encrypted_message, enc_iv)
    print("Decrypted message:", decrypted_message.decode())

    print("TLS/SSL handshake completed")

if __name__ == "__main__":
    main()


Initiating TLS/SSL handshake
Client Hello: b71e8e2d10738dd9bc9f074fcc26c306
Server Hello: 808e92e7d2acf9f9c01d4dd78aa9ebb7
Encrypted Premaster Secret from client: a2c57ac371ef8bec1ffb05081929a7cf57052326982bdb4afb94a1e726602f027c770d5ba88855a105aea873a7b40723b6fd81ec9d56d04fe71266a302634af12e538905e8f6f3b877bd400afdabaf19aacc315f91b7262f0d3ccc765df36e094cc2b8a187628781aa1432a70d49d82f895143d243727faaab150afc627caf54c5e3bf011fbd623c2f7197c2380c3b6b065d77aae112ba5026a6f299a3f349743389b11de7c92260b64cd3a4bec3e8f96e7ab4b7f5f6eb2b666ac9d51888cd9638c00c4e7bfe38810f9dd8ce795419caa2f64f98ad8ceb07d5dc641bd615ce77fa60a245b0875cd75d492cfb3084b13b5ba7cae5db6162baf9d2af7618728796
Session Key: cb9acc5e3cc79a497d9ecefe76687b8aa8df824bdf14105baa0bcda32c85ff62
Client Ready: 7c8ec82bae5f5ea76cfa681a9355871d
Server Ready: 901c9fdd71507ad8f9680d029a53490b
Encrypted message: 4fee5bd25596d618ecc7edd1f00e97d5
Decrypted message: Test Message
TLS/SSL handshake completed
